In [ ]:
%pip install yfinance fredapi pandas matplotlib numpy seaborn plotly

1. VIX Volatility Index - Yahoo Finance (^VIX)

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

def fetch_vix_data(period='1y'):
    """
    Fetch VIX (Volatility Index) data from Yahoo Finance
    
    Parameters:
    period (str): Data period - '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
    
    Returns:
    pandas.DataFrame: VIX data with OHLCV columns
    """
    try:
        # Create ticker object
        vix = yf.Ticker("^VIX")
        
        # Fetch historical data
        vix_data = vix.history(period=period)
        
        # Add calculated fields
        vix_data['Daily_Return'] = vix_data['Close'].pct_change()
        vix_data['Volatility_Level'] = pd.cut(vix_data['Close'], 
                                            bins=[0, 20, 30, 40, float('inf')], 
                                            labels=['Low', 'Moderate', 'High', 'Extreme'])
        
        # Get current value
        current_vix = vix_data['Close'].iloc[-1]
        
        print(f"Current VIX Level: {current_vix:.2f}")
        print(f"Market Fear Level: {vix_data['Volatility_Level'].iloc[-1]}")
        
        return vix_data
        
    except Exception as e:
        print(f"Error fetching VIX  {e}")
        return None

# Usage: vix_data = fetch_vix_data('6mo')


2. Credit Spreads - FRED (BAA10Y, AAA10Y)

In [ ]:
import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

def fetch_credit_spreads(fred_api_key, start_date=None):
    """
    Fetch credit spread data from FRED
    
    Parameters:
    fred_api_key (str): Your FRED API key (get free at https://fred.stlouisfed.org/docs/api/api_key.html)
    start_date (str): Start date in 'YYYY-MM-DD' format, defaults to 2 years ago
    
    Returns:
    pandas.DataFrame: Credit spreads data
    """
    try:
        # Initialize FRED API
        fred = Fred(api_key=fred_api_key)
        
        # Set default start date if not provided
        if start_date is None:
            start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')
        
        # Fetch credit spread series
        series_dict = {
            'BAA_Treasury_Spread': 'BAA10Y',    # Baa Corporate Bond minus 10-Year Treasury
            'AAA_Treasury_Spread': 'AAA10Y',    # Aaa Corporate Bond minus 10-Year Treasury
            'High_Yield_Spread': 'BAMLH0A0HYM2',  # High Yield Option-Adjusted Spread
            'Investment_Grade_Spread': 'BAMLC0A0CM'  # Investment Grade Option-Adjusted Spread
        }
        
        credit_data = pd.DataFrame()
        
        for name, series_id in series_dict.items():
            try:
                data = fred.get_series(series_id, start=start_date)
                credit_data[name] = data
                print(f"✓ Fetched {name}: Current = {data.iloc[-1]:.2f}%")
            except Exception as e:
                print(f"✗ Failed to fetch {name} ({series_id}): {e}")
        
        # Calculate spread differential (Baa - Aaa)
        if 'BAA_Treasury_Spread' in credit_data.columns and 'AAA_Treasury_Spread' in credit_data.columns:
            credit_data['Credit_Quality_Spread'] = (credit_data['BAA_Treasury_Spread'] - 
                                                  credit_data['AAA_Treasury_Spread'])
        
        # Add risk level categorization
        if 'BAA_Treasury_Spread' in credit_data.columns:
            credit_data['Risk_Level'] = pd.cut(credit_data['BAA_Treasury_Spread'],
                                             bins=[0, 1.5, 2.5, 4.0, float('inf')],
                                             labels=['Low Risk', 'Moderate Risk', 'High Risk', 'Crisis'])
        
        return credit_data
        
    except Exception as e:
        print(f"Error fetching credit spreads: {e}")
        return None

# Usage: FRED_API_KEY = "your_key_here"
# credit_data = fetch_credit_spreads(FRED_API_KEY, '2020-01-01')


3. Real Interest Rates - FRED (REAINTRATREARAT10Y)

In [ ]:
import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

def fetch_real_interest_rates(fred_api_key, start_date=None):
    """
    Fetch real interest rates and related data from FRED
    
    Parameters:
    fred_api_key (str): Your FRED API key
    start_date (str): Start date in 'YYYY-MM-DD' format, defaults to 5 years ago
    
    Returns:
    pandas.DataFrame: Real interest rates and related data
    """
    try:
        # Initialize FRED API
        fred = Fred(api_key=fred_api_key)
        
        # Set default start date if not provided
        if start_date is None:
            start_date = (datetime.now() - timedelta(days=1825)).strftime('%Y-%m-%d')
        
        # FRED series for real interest rates and components
        series_dict = {
            'Real_10Y_Rate': 'REAINTRATREARAT10Y',     # 10-Year Real Interest Rate
            'Real_5Y_Rate': 'DFII5',                   # 5-Year TIPS Constant Maturity Rate
            'Nominal_10Y_Rate': 'DGS10',               # 10-Year Treasury Constant Maturity Rate
            'Nominal_5Y_Rate': 'DGS5',                 # 5-Year Treasury Constant Maturity Rate
            'Breakeven_10Y': 'T10YIEM',                # 10-Year Breakeven Inflation Rate
            'Breakeven_5Y': 'T5YIEM',                  # 5-Year Breakeven Inflation Rate
            'Fed_Funds_Rate': 'FEDFUNDS'               # Federal Funds Effective Rate
        }
        
        rates_data = pd.DataFrame()
        
        for name, series_id in series_dict.items():
            try:
                data = fred.get_series(series_id, start=start_date)
                rates_data[name] = data
                if not data.empty:
                    print(f"✓ Fetched {name}: Current = {data.dropna().iloc[-1]:.2f}%")
                else:
                    print(f"⚠ {name}: No data available")
            except Exception as e:
                print(f"✗ Failed to fetch {name} ({series_id}): {e}")
        
        # Calculate derived metrics
        if 'Nominal_10Y_Rate' in rates_data.columns and 'Breakeven_10Y' in rates_data.columns:
            rates_data['Calculated_Real_10Y'] = rates_data['Nominal_10Y_Rate'] - rates_data['Breakeven_10Y']
        
        # Calculate real fed funds rate
        if 'Fed_Funds_Rate' in rates_data.columns and 'Breakeven_5Y' in rates_data.columns:
            rates_data['Real_Fed_Funds'] = rates_data['Fed_Funds_Rate'] - rates_data['Breakeven_5Y']
        
        # Add economic regime classification
        if 'Real_10Y_Rate' in rates_data.columns:
            rates_data['Rate_Regime'] = pd.cut(rates_data['Real_10Y_Rate'],
                                             bins=[-float('inf'), -1, 0, 2, float('inf')],
                                             labels=['Very Negative', 'Negative', 'Low Positive', 'Normal'])
        
        return rates_data
        
    except Exception as e:
        print(f"Error fetching real interest rates: {e}")
        return None

# Usage: real_rates_data = fetch_real_interest_rates(FRED_API_KEY, '2019-01-01')


4. US Dollar Index (DXY) - Yahoo Finance (DX-Y.NYB)

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

def fetch_dxy_data(period='2y'):
    """
    Fetch US Dollar Index (DXY) data from Yahoo Finance
    
    Parameters:
    period (str): Data period - '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
    
    Returns:
    pandas.DataFrame: DXY data with OHLCV and calculated indicators
    """
    try:
        # Create ticker object for DXY
        dxy = yf.Ticker("DX-Y.NYB")
        
        # Fetch historical data
        dxy_data = dxy.history(period=period)
        
        if dxy_data.empty:
            print("No DXY data retrieved. Trying alternative ticker...")
            # Alternative ticker symbols for DXY
            dxy = yf.Ticker("DXY")
            dxy_data = dxy.history(period=period)
        
        if not dxy_data.empty:
            # Calculate technical indicators
            dxy_data['Daily_Return'] = dxy_data['Close'].pct_change()
            dxy_data['SMA_20'] = dxy_data['Close'].rolling(window=20).mean()
            dxy_data['SMA_50'] = dxy_data['Close'].rolling(window=50).mean()
            dxy_data['SMA_200'] = dxy_data['Close'].rolling(window=200).mean()
            
            # Calculate volatility (20-day rolling)
            dxy_data['Volatility_20D'] = dxy_data['Daily_Return'].rolling(window=20).std() * np.sqrt(252) * 100
            
            # Dollar strength categorization
            current_price = dxy_data['Close'].iloc[-1]
            dxy_data['Dollar_Strength'] = pd.cut(dxy_data['Close'],
                                               bins=[0, 90, 95, 100, 105, float('inf')],
                                               labels=['Very Weak', 'Weak', 'Neutral', 'Strong', 'Very Strong'])
            
            # Trend classification based on moving averages
            dxy_data['Trend'] = np.where(
                (dxy_data['Close'] > dxy_data['SMA_20']) & (dxy_data['SMA_20'] > dxy_data['SMA_50']),
                'Uptrend',
                np.where(
                    (dxy_data['Close'] < dxy_data['SMA_20']) & (dxy_data['SMA_20'] < dxy_data['SMA_50']),
                    'Downtrend',
                    'Sideways'
                )
            )
            
            # Get current values
            current_strength = dxy_data['Dollar_Strength'].iloc[-1]
            current_trend = dxy_data['Trend'].iloc[-1]
            
            print(f"Current DXY Level: {current_price:.2f}")
            print(f"Dollar Strength: {current_strength}")
            print(f"Current Trend: {current_trend}")
            print(f"20-Day Volatility: {dxy_data['Volatility_20D'].iloc[-1]:.2f}%")
            
        return dxy_data
        
    except Exception as e:
        print(f"Error fetching DXY  {e}")
        return None

dxy_data = fetch_dxy_data('2y')
dxy_data


Current DXY Level: 98.01
Dollar Strength: Neutral
Current Trend: Sideways
20-Day Volatility: 8.74%


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Return,SMA_20,SMA_50,SMA_200,Volatility_20D,Dollar_Strength,Trend
Date,,,,,,,,,,,,,,
2023-08-14 00:00:00-04:00,102.849998,103.459999,102.769997,103.190002,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Strong,Sideways
2023-08-15 00:00:00-04:00,103.150002,103.269997,102.820000,103.209999,0,0.0,0.0,0.000194,NaN,NaN,NaN,NaN,Strong,Sideways
2023-08-16 00:00:00-04:00,103.209999,103.529999,102.940002,103.419998,0,0.0,0.0,0.002035,NaN,NaN,NaN,NaN,Strong,Sideways
2023-08-17 00:00:00-04:00,103.480003,103.599998,103.059998,103.440002,0,0.0,0.0,0.000193,NaN,NaN,NaN,NaN,Strong,Sideways
2023-08-18 00:00:00-04:00,103.400002,103.680000,103.230003,103.379997,0,0.0,0.0,-0.000580,NaN,NaN,NaN,NaN,Strong,Sideways
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-07 00:00:00-04:00,98.190002,98.470001,97.949997,98.400002,0,0.0,0.0,0.002241,98.400001,98.29200,103.213500,8.791216,Neutral,Uptrend
2025-08-08 00:00:00-04:00,97.980003,98.349998,97.959999,98.180000,0,0.0,0.0,-0.002236,98.416501,98.25800,103.184000,8.815997,Neutral,Sideways
2025-08-11 00:00:00-04:00,98.239998,99.320000,98.029999,98.519997,0,0.0,0.0,0.003463,98.438501,98.24280,103.154450,8.861001,Neutral,Uptrend


Key Thresholds for Alert Systems
VIX Levels:
	•	Below 20: Low volatility/Complacency
	•	20-30: Normal/Moderate volatility
	•	30-40: High volatility/Fear
	•	Above 40: Extreme volatility/Panic
Credit Spreads (Baa-Treasury):
	•	Below 1.5%: Low credit risk
	•	1.5-2.5%: Moderate credit risk
	•	2.5-4.0%: High credit risk
	•	Above 4.0%: Financial stress/Crisis
Real Interest Rates:
	•	Below 0%: Accommodative monetary policy
	•	0-2%: Neutral monetary policy
	•	Above 2%: Restrictive monetary policy
DXY Levels:
	•	Below 95: Weak USD
	•	95-100: Neutral USD
	•	100-105: Strong USD
	•	Above 105: Very Strong USD
API Keys Required
	1.	FRED API Key: Free registration at https://fred.stlouisfed.org/docs/api/api_key.html
	2.	Yahoo Finance: No API key required

Europe and Global: Data sources and Python snippets for a 4‑chart real-time dashboard
Below are Europe- and global-focused, free sources and ready-to-run Python snippets to mirror the four key charts (volatility, credit spreads, real rates, USD/FX). Each snippet uses official or widely supported endpoints outside the US-only context.
1) Volatility (Europe/global): Euro Stoxx 50 VSTOXX and global VIX proxies
	•	Source options:
	•	VSTOXX (EU equity volatility) via Yahoo Finance ticker ^V2TX (unofficial but widely used) .
	•	Global equity proxies: use multiple regional indices if needed; Europe tag pages available in FRED but for equities/vol it’s better via market data feeds; Yahoo Finance remains a free option for EU tickers .
Python (Yahoo Finance, VSTOXX):

In [2]:
import yfinance as yf
import pandas as pd

def fetch_vstoxx(period='1y', interval='1d'):
    tk = yf.Ticker("^V2TX")
    df = tk.history(period=period, interval=interval)
    if df.empty:
        raise RuntimeError("No data for ^V2TX (VSTOXX).")
    df['Daily_Return'] = df['Close'].pct_change()
    return df

# Example
vstoxx = fetch_vstoxx('2y')
vstoxx

HTTP Error 404: 
$^V2TX: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


RuntimeError: No data for ^V2TX (VSTOXX).

2) Credit spreads (Euro area): ECB SDMX API (corporate OAS, gov curves) and OECD cross-country credit indicators
	•	ECB Data Portal offers SDMX API with euro area financial indicators, including corporate bond spread indices and government yields; it’s the official source and supports machine-to-machine access .
	•	OECD API provides cross-country datasets (e.g., Main Economic Indicators) through SDMX; useful for non‑euro economies in Europe and broad global coverage .
Python (ECB SDMX example: 10Y gov yields for Euro area and Germany, then spread):

In [4]:
import pandas as pd

# ECB SDMX REST pattern: https://sdw-wsrest.ecb.europa.eu/service/data/{FLOW}/{KEYS}?{params}
# Example dataset: YC (Yield curve) or FM (Financial markets). We’ll use YC for par yields.
# Dimensions for YC: Yields by country/maturity/type. Example keys can vary; inspect structure first.
# For simplicity, fetch Euro area (U2) and Germany (DE) 10Y par yields (rate type: SR, maturity: 10Y)

def ecb_sdmx_to_df(url):
    # CSV format is supported via ?format=csvdata
    df = pd.read_csv(url)
    # Standardize time/value columns from ECB CSV
    time_col = [c for c in df.columns if c.lower() in ('time', 'time_period')][0]
    value_col = [c for c in df.columns if c.lower() in ('observation value', 'obs_value', 'value')][0]
    out = df[[time_col, value_col]].rename(columns={time_col: 'Date', value_col: 'Value'})
    out['Date'] = pd.to_datetime(out['Date'])
    out = out.set_index('Date').sort_index()
    return out

# Euro area 10Y par yield
ea10_url = ("https://sdw-wsrest.ecb.europa.eu/service/data/YC/"
            "M.U2.EUR.PAR.YC.SR_10Y.A?lastNObservations=1500&format=csvdata")
ea10 = ecb_sdmx_to_df(ea10_url)

# Germany 10Y par yield
de10_url = ("https://sdw-wsrest.ecb.europa.eu/service/data/YC/"
            "M.DE.EUR.PAR.YC.SR_10Y.A?lastNObservations=1500&format=csvdata")
de10 = ecb_sdmx_to_df(de10_url)

spreads = pd.concat([ea10.rename(columns={'Value':'EA10'}),
                     de10.rename(columns={'Value':'DE10'})], axis=1).dropna()
spreads['EA_DE_10Y_Spread_bps'] = (spreads['EA10'] - spreads['DE10']) * 100
# For corporate OAS, explore ECB FM datasets for series keys, then repeat.


HTTPError: HTTP Error 404: 

3) Real interest rates (Europe/global): ECB TIPS/projections and OECD inflation for ex‑US real rate construction
	•	ECB provides inflation, HICP, and market-based inflation expectations; TIPS-equivalent in euro area is limited, but breakevens can be approximated via inflation-linked curves available in ECB data dashboards/APIs .
	•	OECD API supplies CPI/HICP across countries; combine with nominal government yields (ECB for euro area; OECD/BIS for others) to compute ex post “real” rates if market breakevens aren’t available .
Python (Euro area ex‑post real 10Y = nominal 10Y − YoY HICP):

In [ ]:
import pandas as pd

# Euro area 10Y nominal (as above)
ea10 = ecb_sdmx_to_df("https://sdw-wsrest.ecb.europa.eu/service/data/YC/"
                      "M.U2.EUR.PAR.YC.SR_10Y.A?lastNObservations=1500&format=csvdata")

# Euro area HICP YoY (All items) from ECB Data Portal (dataset ICP or PRICES);
# Use HICP annual rate: ICP.M.U2.N.000000.4.ANR (example key; verify structure in portal)
hicp_yoy_url = ("https://sdw-wsrest.ecb.europa.eu/service/data/ICP/"
                "M.U2.N.000000.4.ANR?lastNObservations=1500&format=csvdata")
hicp_yoy = ecb_sdmx_to_df(hicp_yoy_url).rename(columns={'Value':'HICP_YoY'})

# Align monthly HICP YoY with monthly 10Y, compute ex-post real 10Y
df = ea10.rename(columns={'Value':'Nominal10Y'}).join(hicp_yoy, how='inner')
df['ExPostReal10Y'] = df['Nominal10Y'] - df['HICP_YoY']
# df now has Nominal10Y, HICP_YoY, ExPostReal10Y for euro area


4) USD and FX broad dollar vs EUR and trade-weighted alternatives
	•	For global assessments, include EURUSD and the Euro’s trade-weighted index from the ECB Data Portal; ECB publishes effective exchange rates via SDMX .
	•	The DXY index is US-centric; for Europe, monitor EUR nominal effective exchange rate (NEER) and real effective exchange rate (REER) from ECB, plus EURUSD spot via Yahoo Finance for market color .
Python (ECB NEER-20 for euro; Yahoo Finance for EURUSD):

In [3]:
import pandas as pd
import yfinance as yf

# ECB nominal effective exchange rate (NEER-20) for euro area:
# Dataset EXR (exchange rates) with effective rate keys; example key for NEER-20 index (ensure correct key via portal)
neer_url = ("https://sdw-wsrest.ecb.europa.eu/service/data/EXR/"
            "M.U2.NEER20.A?lastNObservations=1500&format=csvdata")
neer = ecb_sdmx_to_df(neer_url).rename(columns={'Value':'EUR_NEER20'})

# EURUSD via Yahoo Finance
eurusd = yf.Ticker("EURUSD=X").history(period="2y")
eurusd = eurusd[['Close']].rename(columns={'Close':'EURUSD'})

# Join on dates (daily vs monthly; you may resample)
eurusd.index = pd.to_datetime(eurusd.index)
combo = neer.join(eurusd.resample('M').last(), how='left')


NameError: name 'ecb_sdmx_to_df' is not defined